In [1]:
!luxai-s2 main.py main.py -v 2 -s 101 -o replay.html

player_0, main.py: Traceback (most recent call last):

player_0, main.py:   File "main.py", line 6, in <module>

player_0, main.py:     from agent import Agent

player_0, main.py:   File "D:\GitHub\lux2022\agent_v4\agent.py", line 7, in <module>

player_0, main.py:     from .actions import unit_should_consider_acting, factory_should_consider_acting, calculate_high_level_unit_action, \

player_0, main.py: ImportError: attempted relative import with no known parent package

player_1, main.py: Traceback (most recent call last):

player_1, main.py:   File "main.py", line 6, in <module>

player_1, main.py:     from agent import Agent

player_1, main.py:   File "D:\GitHub\lux2022\agent_v4\agent.py", line 7, in <module>

player_1, main.py:     from .actions import unit_should_consider_acting, factory_should_consider_acting, calculate_high_level_unit_action, \

player_1, main.py: ImportError: attempted relative import with no known parent package

player_0, main.py: cannot parse action ''
play

Future exception was never retrieved
future: <Future finished exception=BrokenPipeError(32, 'The pipe has been ended', None, 109, None)>
Traceback (most recent call last):
  File "C:\Users\Child\.conda\envs\lux2022\lib\asyncio\proactor_events.py", line 379, in _loop_writing
    f.result()
  File "C:\Users\Child\.conda\envs\lux2022\lib\asyncio\windows_events.py", line 812, in _poll
    value = callback(transferred, key, ov)
  File "C:\Users\Child\.conda\envs\lux2022\lib\asyncio\windows_events.py", line 538, in finish_send
    return ov.getresult()
BrokenPipeError: [WinError 109] The pipe has been ended
Future exception was never retrieved
future: <Future finished exception=BrokenPipeError(32, 'The pipe has been ended', None, 109, None)>
Traceback (most recent call last):
  File "C:\Users\Child\.conda\envs\lux2022\lib\asyncio\proactor_events.py", line 379, in _loop_writing
    f.result()
  File "C:\Users\Child\.conda\envs\lux2022\lib\asyncio\windows_events.py", line 812, in _poll
    val

# Just double click replay.html to open window in jupyter lab
This is just a note for above. The replays work well in tabls li

In [2]:
# !pip install --upgrade luxai_s2
# !pip install importlib-metadata==4.13.0
# !pip install --upgrade moviepy # needed to render videos of episodes
# !pip install opencv-python


In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
print('\n'.join(sys.path))

D:\GitHub\lux2022\agent_v4
C:\Users\Child\.conda\envs\lux2022\python38.zip
C:\Users\Child\.conda\envs\lux2022\DLLs
C:\Users\Child\.conda\envs\lux2022\lib
C:\Users\Child\.conda\envs\lux2022

C:\Users\Child\AppData\Roaming\Python\Python38\site-packages
C:\Users\Child\.conda\envs\lux2022\lib\site-packages
C:\Users\Child\.conda\envs\lux2022\lib\site-packages\win32
C:\Users\Child\.conda\envs\lux2022\lib\site-packages\win32\lib
C:\Users\Child\.conda\envs\lux2022\lib\site-packages\Pythonwin
D:\GitHub\lux2022


In [4]:
from luxai_s2.env import LuxAI_S2
import matplotlib.pyplot as plt
import numpy as np
from agent_v4.agent import Agent

ImportError: cannot import name 'GeneralObs' from partially initialized module 'agent_v4.agent' (most likely due to a circular import) (D:\GitHub\lux2022\agent_v4\agent.py)

In [ ]:
def animate(imgs, _return=True):
    # using cv2 to generate videos as moviepy doesn't work on kaggle notebooks
    import cv2
    import os
    import string
    import random
    video_name = ''.join(random.choice(string.ascii_letters) for i in range(18))+'.webm'
    height, width, layers = imgs[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'VP90')
    video = cv2.VideoWriter(video_name, fourcc, 10, (width,height))

    for img in imgs:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        video.write(img)
    video.release()
    if _return:
        from IPython.display import Video
        return Video(video_name)
def interact(env, agents, steps):
    # reset our env
    obs = env.reset(seed=41)
    np.random.seed(0)
    imgs = []
    step = 0
    # Note that as the environment has two phases, we also keep track a value called 
    # `real_env_steps` in the environment state. The first phase ends once `real_env_steps` is 0 and used below

    # iterate until phase 1 ends
    while env.state.real_env_steps < 0:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].early_setup(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
    done = False
    while not done:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].act(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
        done = dones["player_0"] and dones["player_1"]
    return animate(imgs)

We can now create an environment and start interacting with it, as well as look at what the observation is like

In [ ]:
env = LuxAI_S2() # create the environment object
obs = env.reset(seed=41) # resets an environment with a seed

In [ ]:
# the observation is always composed of observations for both players.
obs.keys(), obs["player_0"].keys()

In [ ]:
# visualize the environment so far with rgb_array to get a quick look at the map
# dark orange - high rubble, light orange - low rubble
# blue = ice, yellow = ore
img = env.render("rgb_array", width=640, height=640)
plt.imshow(img)

In [ ]:
# recreate our agents and run
agents = {player: Agent(player, env.state.env_cfg) for player in env.agents}
interact(env, agents, 25)

## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

In [ ]:
#!tar -czf submission.tar.gz *

## Submit
Now open the /kaggle/working folder and find submission.tar.gz, download that file, navigate to the "MySubmissions" tab in https://www.kaggle.com/c/lux-ai-season-2/ and upload your submission! It should play a validation match against itself and once it succeeds it will be automatically matched against other players' submissions. Newer submissions will be prioritized for games over older ones. Your team is limited in the number of succesful submissions per day so we highly recommend testing your bot locally before submitting.